# Задача регрессии

In [27]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn import linear_model
from geopy.distance import vincenty
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns

## Первичный анализ данных и подготовка данных
### Интеграция данных

In [28]:
df_vacancy = pd.read_csv('hh_vacancy.csv')
df_vacancy

,id,area_id,area_name,employer_id,employment_id,employment_name,experience_id,experience_name,key_skills,name,published_at,salary_currency,salary_from,salary_to,schedule_id,schedule_name,specialization_id,specialization_name
0,25607531,1,Москва,1747017.0,full,Полная занятость,between3And6,От 3 до 6 лет,"PHP, Symfony, PostgreSQL, Redis, Design Patter...",Senior/Middle PHP-Developer,2018-05-06T17:43:45+0300,RUR,120000.0,170000.0,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"
1,25435136,1,Москва,553714.0,project,Проектная работа,between3And6,От 3 до 6 лет,NaN,Ведущий фронтенд-разработчик,2018-05-05T14:43:01+0300,NaN,NaN,NaN,remote,Удаленная работа,1,"Информационные технологии, интернет, телеком"
2,25284500,84,Ставрополь,5402.0,full,Полная занятость,between3And6,От 3 до 6 лет,"JavaScript, HTML5, CSS3, AngularJS, Bootstrap",Frontend-разработчик,2018-05-04T09:24:18+0300,RUR,60000.0,80000.0,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"
3,25641559,1,Москва,1554.0,full,Полная занятость,between1And3,От 1 года до 3 лет,"1С программирование, 1С: Бухгалтерия, 1С Управ...",Программист-консультант 1С,2018-05-06T10:49:05+0300,NaN,NaN,NaN,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"
4,25527035,2,Санкт-Петербург,3289100.0,full,Полная занятость,between1And3,От 1 года до 3 лет,NaN,Менеджер по продажам,2018-05-05T16:15:16+0300,RUR,35000.0,100000.0,flexible,Гибкий график,1,"Информационные технологии, интернет, телеком"
5,25178591,1,Москва,2353123.0,full,Полная занятость,between3And6,От 3 до 6 лет,NaN,Системный аналитик,2018-05-07T00:54:39+0300,NaN,NaN,NaN,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"
6,25449125,1,Москва,934019.0,full,Полная занятость,between3And6,От 3 до 6 лет,NaN,Java-разработчик,2018-05-06T11:41:28+0300,RUR,120000.0,220000.0,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"
7,25459894,1,Москва,1053428.0,full,Полная занятость,between3And6,От 3 до 6 лет,"Linux, AWS, Proxmox, Vagrant, Docker, Админист...",Системный администратор (DevOps),2018-05-06T17:00:04+0300,RUR,100000.0,NaN,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"
8,25041175,2,Санкт-Петербург,934019.0,full,Полная занятость,between1And3,От 1 года до 3 лет,NaN,Java разработчик (middle/senior),2018-05-05T18:43:16+0300,RUR,120000.0,220000.0,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"
9,25295125,78,Самара,2618807.0,full,Полная занятость,between1And3,От 1 года до 3 лет,"Коммуникабельность, Аналитический склад ума, Н...",Менеджер по работе с корпоративными клиентами,2018-05-06T11:45:27+0300,RUR,35000.0,NaN,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"


In [29]:
df_vacancy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19837 entries, 0 to 19836
Data columns (total 18 columns):
id                     19837 non-null int64
area_id                19837 non-null int64
area_name              19837 non-null object
employer_id            19813 non-null float64
employment_id          19805 non-null object
employment_name        19805 non-null object
experience_id          19805 non-null object
experience_name        19805 non-null object
key_skills             6654 non-null object
name                   19837 non-null object
published_at           19837 non-null object
salary_currency        10358 non-null object
salary_from            9611 non-null float64
salary_to              4594 non-null float64
schedule_id            19805 non-null object
schedule_name          19805 non-null object
specialization_id      19837 non-null int64
specialization_name    19837 non-null object
dtypes: float64(3), int64(3), object(12)
memory usage: 2.7+ MB


In [30]:
# Разделение на обучающую и тестовую выборку по id, который соответствует порядку добавления вакаансий на сайт.
count_df = len(df_vacancy)
df_vacancy.sort_values(by='id', ascending=True, inplace=True)
df_vacancy_test = df_vacancy[int(2*count_df/3):]
df_vacancy_train = df_vacancy[:int(2*count_df/3)]

In [31]:
# Первичный анализ данных и формирование признаков рекомендуется проводить на обучающей выборке.
df_vacancy_train.describe()

,id,area_id,employer_id,salary_from,salary_to,specialization_id
count,1.322400e+04,13224.000000,1.321900e+04,6555.000000,2695.000000,13224.000000
mean,2.427900e+07,339.665910,6.285927e+05,52182.384592,85212.069388,3.595886
std,1.580547e+06,800.582578,8.361471e+05,35127.270832,59299.426350,4.907250
min,7.160726e+06,1.000000,1.900000e+01,25.000000,30.000000,1.000000
25%,2.405097e+07,1.000000,4.114400e+04,30000.000000,40000.000000,1.000000
50%,2.489626e+07,17.000000,8.458500e+04,43000.000000,70000.000000,1.000000
75%,2.524230e+07,88.000000,9.994420e+05,64000.000000,120000.000000,4.000000
max,2.540689e+07,5962.000000,3.353801e+06,300000.000000,1000000.000000,29.000000


In [32]:
# Удаление записей с пропущенными значениями NaN.
df_vacancy_train = df_vacancy_train[np.isfinite(df_vacancy_train['salary_from']) &
                                    np.isfinite(df_vacancy_train['area_id']) &
                                    np.isfinite(df_vacancy_train['employer_id']) &
                                    np.isfinite(df_vacancy_train['specialization_id'])]

In [33]:
df_company = pd.read_csv('hh_company.csv')
df_company.head()

,employer_id,employer_name,employer_site,industry_id,industry_name
0,1747017.0,Колтач Солюшнс,NaN,7,"Информационные технологии, системная интеграци..."
1,553714.0,Нотамедиа,NaN,7,"Информационные технологии, системная интеграци..."
2,5402.0,НЕОЛАНТ,NaN,7,"Информационные технологии, системная интеграци..."
3,1554.0,Infosuite,NaN,7,"Информационные технологии, системная интеграци..."
4,3289100.0,Медиа Бокс,NaN,7,"Информационные технологии, системная интеграци..."


In [34]:
df_vacancy_train = pd.merge(df_vacancy_train, df_company, on='employer_id')
df_vacancy_train.head()

,id,area_id,area_name,employer_id,employment_id,employment_name,experience_id,experience_name,key_skills,name,...,salary_from,salary_to,schedule_id,schedule_name,specialization_id,specialization_name,employer_name,employer_site,industry_id,industry_name
0,7160726,2,Санкт-Петербург,558625.0,part,Частичная занятость,noExperience,Нет опыта,NaN,Оператор call-центра,...,35000.0,NaN,flexible,Гибкий график,3,"Маркетинг, реклама, PR",Cистема Северо-Запад,NaN,7,"Информационные технологии, системная интеграци..."
1,7160726,2,Санкт-Петербург,558625.0,part,Частичная занятость,noExperience,Нет опыта,NaN,Оператор call-центра,...,35000.0,NaN,flexible,Гибкий график,3,"Маркетинг, реклама, PR",Cистема Северо-Запад,NaN,7,"Информационные технологии, системная интеграци..."
2,7160726,2,Санкт-Петербург,558625.0,part,Частичная занятость,noExperience,Нет опыта,NaN,Оператор call-центра,...,35000.0,NaN,flexible,Гибкий график,3,"Маркетинг, реклама, PR",Cистема Северо-Запад,NaN,7,"Информационные технологии, системная интеграци..."
3,7160726,2,Санкт-Петербург,558625.0,part,Частичная занятость,noExperience,Нет опыта,NaN,Оператор call-центра,...,35000.0,NaN,flexible,Гибкий график,3,"Маркетинг, реклама, PR",Cистема Северо-Запад,NaN,7,"Информационные технологии, системная интеграци..."
4,7160726,2,Санкт-Петербург,558625.0,part,Частичная занятость,noExperience,Нет опыта,NaN,Оператор call-центра,...,35000.0,NaN,flexible,Гибкий график,3,"Маркетинг, реклама, PR",Cистема Северо-Запад,NaN,7,"Информационные технологии, системная интеграци..."


In [35]:
df_cities = pd.read_csv('cities.csv')
df_cities.head()

,Индекс,Тип региона,Регион,Тип района,Район,Тип города,Город,Тип н/п,Н/п,Код КЛАДР,...,Уровень по ФИАС,Признак центра района или региона,Код ОКАТО,Код ОКТМО,Код ИФНС,Часовой пояс,Широта,Долгота,Федеральный округ,Население
0,385200.0,Респ,Адыгея,NaN,NaN,г,Адыгейск,NaN,NaN,100000200000,...,4: город,0,79403000000,7.970300e+10,107,UTC+3,44.878372,39.190172,Южный,12689
1,385000.0,Респ,Адыгея,NaN,NaN,г,Майкоп,NaN,NaN,100000100000,...,4: город,2,79401000000,7.970100e+10,105,UTC+3,44.609827,40.100653,Южный,144055
2,649000.0,Респ,Алтай,NaN,NaN,г,Горно-Алтайск,NaN,NaN,400000100000,...,4: город,2,84401000000,8.470100e+07,400,UTC+7,51.958268,85.960296,Сибирский,62861
3,658125.0,край,Алтайский,NaN,NaN,г,Алейск,NaN,NaN,2200000200000,...,4: город,0,1403000000,1.703000e+06,2201,UTC+7,52.492091,82.779415,Сибирский,28528
4,656000.0,край,Алтайский,NaN,NaN,г,Барнаул,NaN,NaN,2200000100000,...,4: город,2,1401000000,1.701000e+06,2200,UTC+7,53.348115,83.779836,Сибирский,635585


In [36]:
df_cities = df_cities[['Город','Широта','Долгота','Население', 'Регион']]
df_cities['Город'] = df_cities.apply(lambda row: row['Регион']
                                     if type(row['Город']) != str and np.isnan(row['Город'])
                                     else row['Город'], axis=1)
df_cities = df_cities[['Город','Широта','Долгота','Население']]
df_cities = df_cities.rename(index=str, columns={"Город": "area_name"})
df_cities = df_cities.rename(index=str, columns={"Широта": "latitude"})
df_cities = df_cities.rename(index=str, columns={"Долгота": "longitude"})
df_cities = df_cities.rename(index=str, columns={"Население": "population"})

In [37]:
moscow_latitude, moscow_longitude = list(df_cities.loc[df_cities.area_name == 'Москва'][['latitude','longitude']].iloc[0])
moscow_latitude, moscow_longitude

(55.7538789, 37.6203735)

In [38]:
df_cities['distance_to_Moscow'] = df_cities.apply(lambda row: vincenty((moscow_latitude, moscow_longitude),
                                                                       (row.latitude, row.longitude)).kilometers, axis=1)
df_cities.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning:

Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.



,area_name,latitude,longitude,population,distance_to_Moscow
0,Адыгейск,44.878372,39.190172,12689,1214.804843
1,Майкоп,44.609827,40.100653,144055,1251.957644
2,Горно-Алтайск,51.958268,85.960296,62861,3142.838553
3,Алейск,52.492091,82.779415,28528,2921.575344
4,Барнаул,53.348115,83.779836,635585,2943.040433


In [39]:
df_vacancy_train = pd.merge(df_vacancy_train, df_cities, on='area_name')
df_vacancy_train.head()

,id,area_id,area_name,employer_id,employment_id,employment_name,experience_id,experience_name,key_skills,name,...,specialization_id,specialization_name,employer_name,employer_site,industry_id,industry_name,latitude,longitude,population,distance_to_Moscow
0,7160726,2,Санкт-Петербург,558625.0,part,Частичная занятость,noExperience,Нет опыта,NaN,Оператор call-центра,...,3,"Маркетинг, реклама, PR",Cистема Северо-Запад,NaN,7,"Информационные технологии, системная интеграци...",59.939125,30.315822,4848742,636.038853
1,7160726,2,Санкт-Петербург,558625.0,part,Частичная занятость,noExperience,Нет опыта,NaN,Оператор call-центра,...,3,"Маркетинг, реклама, PR",Cистема Северо-Запад,NaN,7,"Информационные технологии, системная интеграци...",59.939125,30.315822,4848742,636.038853
2,7160726,2,Санкт-Петербург,558625.0,part,Частичная занятость,noExperience,Нет опыта,NaN,Оператор call-центра,...,3,"Маркетинг, реклама, PR",Cистема Северо-Запад,NaN,7,"Информационные технологии, системная интеграци...",59.939125,30.315822,4848742,636.038853
3,7160726,2,Санкт-Петербург,558625.0,part,Частичная занятость,noExperience,Нет опыта,NaN,Оператор call-центра,...,3,"Маркетинг, реклама, PR",Cистема Северо-Запад,NaN,7,"Информационные технологии, системная интеграци...",59.939125,30.315822,4848742,636.038853
4,7160726,2,Санкт-Петербург,558625.0,part,Частичная занятость,noExperience,Нет опыта,NaN,Оператор call-центра,...,3,"Маркетинг, реклама, PR",Cистема Северо-Запад,NaN,7,"Информационные технологии, системная интеграци...",59.939125,30.315822,4848742,636.038853


In [40]:
for column in df_vacancy_train.columns:
    print(column)
    values = df_vacancy_train[column].dropna().unique()
    sorted_values = sorted(values.tolist())
    print(len(sorted_values))
    if len(sorted_values) < 20:
        print(f"{sorted_values}")
    else:
        print(f"[{min(sorted_values)}: {max(sorted_values)}]")
    inter = set(df_vacancy_train[column].unique()) - set(df_vacancy_train[column].dropna().unique())
    if inter:
        print(inter)
    print()

id
6151
[7160726: 25406869]

area_id
337
[1: 4455]

area_name
337
[Абакан: Яхрома]

employer_id
1349
[139.0: 3351855.0]

employment_id
4
['full', 'part', 'probation', 'project']

employment_name
4
['Полная занятость', 'Проектная работа', 'Стажировка', 'Частичная занятость']

experience_id
4
['between1And3', 'between3And6', 'moreThan6', 'noExperience']

experience_name
4
['Более 6 лет', 'Нет опыта', 'От 1 года до 3 лет', 'От 3 до 6 лет']

key_skills
1626
[.NET Framework, ASP.NET, C#, Entity Framework, MVC, LINQ, JavaScript, HTML5, CSS3, MS SQL Server, Angular, WebAPI: № ФЗ-223, №ФЗ-44, B2B Продажи, Ведение переговоров, Заключение договоров, Поиск и привлечение клиентов]
{nan}

name
2590
[.NEТ/C# Developer: Юрист по международному праву]

published_at
4077
[2018-04-07T10:01:09+0300: 2018-05-07T05:51:25+0300]

salary_currency
4
['EUR', 'RUR', 'UAH', 'USD']

salary_from
178
[25.0: 300000.0]

salary_to
129
[30.0: 1000000.0]
{nan}

schedule_id
5
['flexible', 'flyInFlyOut', 'fullDay', 'remote

In [41]:
categorial_factors = ['area_name',
                      'employment_name',
                      'salary_currency',
                      'schedule_name',
                      'specialization_id'
                  ]
quality_factors = ['latitude',
                   'longitude',
                   'population',
                   'distance_to_Moscow'
                  ]

In [42]:
df_vacancy_train.describe()

,id,area_id,employer_id,salary_from,salary_to,specialization_id,employer_site,industry_id,latitude,longitude,distance_to_Moscow
count,4.495943e+06,4.495943e+06,4.495943e+06,4.495943e+06,393001.000000,4.495943e+06,0.0,4495943.0,4.495943e+06,4.495943e+06,4.495943e+06
mean,2.407015e+07,1.229713e+03,1.092549e+05,4.999383e+04,124056.618418,5.353185e+00,NaN,7.0,5.499234e+01,4.324840e+01,5.374303e+02
std,1.677781e+06,9.493956e+02,2.237704e+05,2.081256e+04,45219.070170,3.701526e+00,NaN,0.0,3.206124e+00,1.479496e+01,9.083851e+02
min,7.160726e+06,1.000000e+00,1.390000e+02,2.500000e+01,30.000000,1.000000e+00,NaN,7.0,4.282404e+01,2.050725e+01,0.000000e+00
25%,2.320238e+07,7.200000e+01,7.863800e+04,4.000000e+04,90000.000000,4.000000e+00,NaN,7.0,5.483252e+01,3.748310e+01,5.462097e+01
50%,2.492785e+07,1.793000e+03,7.863800e+04,5.000000e+04,150000.000000,5.000000e+00,NaN,7.0,5.571703e+01,3.821940e+01,1.251301e+02
75%,2.509473e+07,2.054000e+03,7.863800e+04,6.200000e+04,150000.000000,5.000000e+00,NaN,7.0,5.612816e+01,4.131696e+01,7.054913e+02
max,2.540687e+07,4.455000e+03,3.351855e+06,3.000000e+05,1000000.000000,2.600000e+01,NaN,7.0,6.907650e+01,1.586559e+02,6.796347e+03


In [44]:
df_vacancy_train.columns

Index(['id', 'area_id', 'area_name', 'employer_id', 'employment_id',
       'employment_name', 'experience_id', 'experience_name', 'key_skills',
       'name', 'published_at', 'salary_currency', 'salary_from', 'salary_to',
       'schedule_id', 'schedule_name', 'specialization_id',
       'specialization_name', 'employer_name', 'employer_site', 'industry_id',
       'industry_name', 'latitude', 'longitude', 'population',
       'distance_to_Moscow'],
      dtype='object')

In [ ]:
#Приводим к одной валюте
def valute(salary, currency):
    if currency == 'USD':
        return salary*65
    elif currency == 'EUR':
        return salary*74
    return salary

print(df_vacancy_train['salary_currency'].unique())
df_vacancy_train['salary_from'] = df_vacancy_train.apply(lambda row: valute(row.salary_from, row.salary_currency), axis=1)

['RUR' 'EUR' 'USD' 'UAH']


In [ ]:
areas = df_vacancy_train[['id', 'area_name']].\
        groupby(['area_name']).\
        agg({'id': 'count'}).\
        rename(columns={'id': 'area_vacancy_count'})
areas.head()

In [ ]:
areas = df_vacancy_train[['salary_from', 'area_name']].\
        groupby(['area_name']).\
        agg({'salary_from': 'mean'}).\
        rename(columns={'salary_from': 'area_salary_mean'})
areas.head()

### Графический анализ

In [17]:
vis_data = df_vacancy_train[quality_factors + ['salary_from']].sample(frac=0.001)
sns.pairplot(vis_data, hue='salary_from')

ValueError: setting an array element with a sequence

AttributeError: 'NoneType' object has no attribute 'seq'

In [ ]:
for factor in categorial_factors:
    print(factor)
    g = sns.factorplot(factor,'salary_from',data=df_vacancy_train.sample(frac=0.001), aspect=2.5)
    g.set_xticklabels(rotation=0)
    plt.show()

### Формирование признаков

In [9]:
train_y = df_vacancy_train['salary_from']
train_X = df_vacancy_train[['area_id', 'employer_id', 'specialization_id']]

## Обучение модели

In [10]:
reg = linear_model.LinearRegression()
reg.fit(train_X, train_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [11]:
# Коэф. уравнения.
reg.coef_

array([-2.44679546e+00, -3.34828989e-03, -1.54451998e+03])

## Оценка качества обучения

In [12]:
# Точность на обучающей выборке.
train_r2 = r2_score(train_y, reg.predict(train_X))
train_r2

0.06585492603413479

In [13]:
# Подготовка тестовой выборки.
df_vacancy_test = df_vacancy_test[np.isfinite(df_vacancy_test['salary_from']) &
                                  np.isfinite(df_vacancy_test['area_id']) &
                                  np.isfinite(df_vacancy_test['employer_id']) &
                                  np.isfinite(df_vacancy_test['specialization_id'])]
test_y = df_vacancy_test['salary_from']
test_X = df_vacancy_test[['area_id', 'employer_id', 'specialization_id']]
# Точность на тестовой выборке.
test_r2 = r2_score(test_y, reg.predict(test_X))
test_r2

0.021286993703545587

In [17]:
def adjusted_r2_score(test_y, test_X, predict_y):
    '''Коэффициент детерминации, скорректированный на число признаков.'''
    n=test_X.shape[0]       # количество наблюдений
    p=test_X.shape[1] - 1   # количество признаков, включенных в модель
    r2 = r2_score(test_y, predict_y)
    adj_r2 = 1 - (1 - r2) * ((n - 1)/(n-p-1))
    return adj_r2

# Скорректированная точность на тестовой выборке.
test_adj_r2 = adjusted_r2_score(test_y, test_X, reg.predict(test_X))
test_adj_r2

0.02064289169216249

Качество модели очень плохое, поскольку признаков мало и использован простейший способ кодирования категорильнаых признаков, который не подходит для линейны модели. Необходимиый результат коэф. детерминации 0.7